In [9]:
# Load the model that we created in Part 2
from gensim.models import Word2Vec
model = Word2Vec.load("300features_40minwords_10context")   # type: Word2Vec

2017-08-29 13:08:53,856 : INFO : loading Word2Vec object from 300features_40minwords_10context


2017-08-29 13:08:54,017 : INFO : loading wv recursively from 300features_40minwords_10context.wv.* with mmap=None


2017-08-29 13:08:54,019 : INFO : setting ignored attribute syn0norm to None


2017-08-29 13:08:54,019 : INFO : setting ignored attribute cum_table to None


2017-08-29 13:08:54,020 : INFO : loaded 300features_40minwords_10context


In [25]:
import pandas as pd

# Read data from files 
train = pd.read_csv( "labeledTrainData.tsv", header=0, delimiter="\t", quoting=3 )
test = pd.read_csv( "testData.tsv", header=0, delimiter="\t", quoting=3 )
unlabeled_train = pd.read_csv( "unlabeledTrainData.tsv", header=0, delimiter="\t", quoting=3)

# Verify the number of reviews that were read (100,000 in total)
print "Read %d labeled train reviews, %d labeled test reviews, " \
      "and %d unlabeled reviews\n" % (train["review"].size,  
 test["review"].size, unlabeled_train["review"].size )

Read 25000 labeled train reviews, 25000 labeled test reviews, and 50000 unlabeled reviews



In [21]:
# Import various modules for string cleaning
from bs4 import BeautifulSoup
import re
from nltk.corpus import stopwords


def review_to_wordlist(review, remove_stopwords=False):
    # Function to convert a document to a sequence of words,
    # optionally removing stop words.  Returns a list of words.
    
    # 1. Remove HTML
    review_text = BeautifulSoup(review, 'lxml').get_text()
    
    # 2. Remove non-letters
    review_text = re.sub(r'[^\x00-\x7F]', ' ', review_text)
    review_text = re.sub("[^a-zA-Z]"," ", review_text)
    
    # 3. Convert words to lower case and split them
    words = review_text.lower().split()

    # 4. Optionally remove stop words (false by default)
    if remove_stopwords:
        stops = set(stopwords.words("english"))
        words = [w for w in words if not w in stops]

    # 5. Return a list of words
    return words

In [20]:
print model.wv.syn0.shape

(16490, 300)


In [31]:
import numpy as np  # Make sure that numpy is imported


def makeFeatureVec(words, model, num_features):
    # Function to average all of the word vectors in a given
    # paragraph
    #
    # Pre-initialize an empty numpy array (for speed)
    featureVec = np.zeros((num_features,),dtype="float32")
    #
    nwords = 0.
    # 
    # Index2word is a list that contains the names of the words in 
    # the model's vocabulary. Convert it to a set, for speed 
    index2word_set = set(model.wv.index2word)
    #
    # Loop over each word in the review and, if it is in the model's
    # vocaublary, add its feature vector to the total
    for word in words:
        if word in index2word_set: 
            nwords = nwords + 1.
            featureVec = np.add(featureVec,model[word])
    # 
    # Divide the result by the number of words to get the average
    featureVec = np.divide(featureVec,nwords)
    return featureVec


def getAvgFeatureVecs(reviews, model, num_features):
    # Given a set of reviews (each one a list of words), calculate 
    # the average feature vector for each one and return a 2D numpy array 
    # 
    # Initialize a counter
    counter = 0.
    # 
    # Preallocate a 2D numpy array, for speed
    reviewFeatureVecs = np.zeros((len(reviews),num_features),dtype="float32")
    # 
    # Loop through the reviews
    for review in reviews:
		#
		# Print a status message every 1000th review
		if counter%1000. == 0.:
		   print "Review %d of %d" % (counter, len(reviews))
		# 
		# Call the function (defined above) that makes average feature vectors
		reviewFeatureVecs[int(counter)] = makeFeatureVec(review, model, num_features)
		#
		# Increment the counter
		counter = counter + 1.
    return reviewFeatureVecs

In [32]:
# ****************************************************************
# Calculate average feature vectors for training and testing sets,
# using the functions we defined above. Notice that we now use stop word
# removal.

clean_train_reviews = []
for review in train["review"]:
    clean_train_reviews.append(review_to_wordlist(review, remove_stopwords=True))

trainDataVecs = getAvgFeatureVecs(clean_train_reviews, model, num_features)

print "Creating average feature vecs for test reviews"
clean_test_reviews = []
for review in test["review"]:
    clean_test_reviews.append(review_to_wordlist(review, remove_stopwords=True))

testDataVecs = getAvgFeatureVecs(clean_test_reviews, model, num_features)

Review 0 of 25000


Review 1000 of 25000


Review 2000 of 25000


Review 3000 of 25000


Review 4000 of 25000


...

Review 20000 of 25000


Review 21000 of 25000


Review 22000 of 25000


Review 23000 of 25000


Review 24000 of 25000


In [53]:
# fill in nan to 0
count = 0
for i in range(len(trainDataVecs)):
	if np.all(np.isnan(trainDataVecs[i])):
		for j in range(len(trainDataVecs[i])):
			trainDataVecs[i][j] = 0
			
for i in range(len(testDataVecs)):
	if np.any(np.isnan(testDataVecs[i])):
		for j in range(len(testDataVecs[i])):
			if np.isnan(testDataVecs[i][j]):
				testDataVecs[i][j] = 0

print np.any(np.isnan(trainDataVecs))
print np.any(np.isnan(testDataVecs))

False
False


In [54]:
# Fit a random forest to the training data, using 100 trees
from sklearn.ensemble import RandomForestClassifier
forest = RandomForestClassifier(n_estimators = 100)

print "Fitting a random forest to labeled training data..."
forest = forest.fit(trainDataVecs, train["sentiment"])

# Test & extract results 
result = forest.predict(testDataVecs)

# Write the test results 
output = pd.DataFrame(data={"id":test["id"], "sentiment":result})
output.to_csv("Word2Vec_AverageVectors.csv", index=False, quoting=3)

Fitting a random forest to labeled training data...
